In [3]:
import os
os.environ["PJRT_DEVICE"] = "CPU"


In [4]:
import torch
from torch import nn
from typing import Dict, Tuple
from torch.nn import Linear, LayerNorm, TransformerEncoder, TransformerEncoderLayer
import torch.nn.functional as F
from einops import rearrange
import math
import ai_edge_torch  # Ensure AI Edge Torch is installed


class TransformerEncoderWAttention(nn.TransformerEncoder):
    def forward(self, src, mask=None, src_key_padding_mask=None):
        output = src
        # Make sure there are no mutable state changes by not storing attention weights
        for layer in self.layers:
            
            output = layer(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
        return output

class TransModel(nn.Module):
    def __init__(self,
                 mocap_frames=128,
                 num_joints=32,
                 acc_frames=128,
                 num_classes=2,     
                 num_heads=2,
                 acc_coords=4,
                 av=False,
                 num_layer=2,
                 norm_first=True,
                 embed_dim=32,     
                 activation='relu',
                 **kwargs):
        super().__init__()
        self.data_shape = (acc_frames, acc_coords)
        self.length = self.data_shape[0]  # 128
        size = self.data_shape[1]         # 4

       
        self.input_proj = nn.Sequential(
            nn.Conv1d(size, embed_dim, kernel_size=3, stride=1, padding='same'),          
            nn.Conv1d(embed_dim, embed_dim * 2, kernel_size=3, stride=1, padding='same'), 
            nn.Conv1d(embed_dim * 2, embed_dim, kernel_size=3, stride=1, padding='same')  
        )

        
        self.encoder_layer = TransformerEncoderLayer(
            d_model=self.length,          # 128
            nhead=num_heads,              # 2
            dim_feedforward=32,
            activation=activation,
            dropout=0.5,
            batch_first=False,
            norm_first=norm_first
        )

        
        self.encoder = TransformerEncoderWAttention(
            encoder_layer=self.encoder_layer,
            num_layers=num_layer,
            norm=nn.LayerNorm(embed_dim)  # LayerNorm over embed_dim (32)
        )

        
        self.ln1 = nn.Linear(self.length, 32)  # 128 -> 32
        self.ln2 = nn.Linear(32, 16)
        self.drop2 = nn.Dropout(p=0.5)
        self.output = Linear(16, num_classes)  # 16 -> 2
        nn.init.normal_(self.output.weight, 0, math.sqrt(2. / num_classes))

    def forward(self, acc_data, skl_data):
        b, l, c = acc_data.shape  # b, 128, 4
        x = rearrange(acc_data, 'b l c -> b c l')  # [b, 4, 128]
        x = self.input_proj(x)                     # [b, 32, 128]
        x = rearrange(x, 'b c l -> c b l')         # [32, b, 128]
        x = self.encoder(x)                        # [32, b, 128]
        x = rearrange(x, 'c b l -> b l c')         # [b, 128, 32]

        
        x = F.avg_pool1d(x, kernel_size=x.shape[-1], stride=1)  # [b, 128, 1]
        x = x.view(b, -1)                          # [b, 128]
        x = F.relu(self.ln1(x))                    # [b, 32]
        x = self.drop2(x)
        x = F.relu(self.ln2(x))                    # [b, 16]
        x = self.output(x)                         # [b, num_classes]
        return x


def load_and_convert_model(pth_file_path, tflite_file_path):
    
    model = TransModel()
    model.eval()

    
    checkpoint = torch.load(pth_file_path, map_location='cpu')

    
    model.load_state_dict(checkpoint, strict=True)

    
    class ModelWrapper(nn.Module):
        def __init__(self):
            super().__init__()
            self.model = model

        def forward(self, acc_data):
            skl_data_dummy = torch.zeros(acc_data.shape[0], 128, 32, 3)
            return self.model(acc_data, skl_data_dummy)

    wrapped_model = ModelWrapper()
    wrapped_model.eval()
    wrapped_model.to('cpu')

    
    acc_data_sample = torch.randn(1, 128, 4)

   
   
    edge_model = ai_edge_torch.convert(
        wrapped_model,
        (acc_data_sample,)
    )

    
    edge_model.export(tflite_file_path)
    print(f"Model successfully converted to TFLite format and saved as '{tflite_file_path}'.")

if __name__ == "__main__":
    # Path to your saved .pth file
    pth_file_path = "ttfWeights.pth"  
    tflite_file_path = "transmodel_converted1.tflite"

    
    load_and_convert_model(pth_file_path, tflite_file_path)


/home/abheekp/ai_edge_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/tmp/ipykernel_36651/846237639.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case

INFO:tensorflow:Assets written to: /tmp/tmpjo2y6ts0/assets


INFO:tensorflow:Assets written to: /tmp/tmpjo2y6ts0/assets
W0000 00:00:1730397276.881676   36651 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1730397276.881733   36651 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-10-31 12:54:36.882774: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjo2y6ts0
2024-10-31 12:54:36.883570: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-31 12:54:36.883584: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpjo2y6ts0
I0000 00:00:1730397276.893503   36651 mlir_graph_optimization_pass.cc:360] MLIR V1 optimization pass is not enabled
2024-10-31 12:54:36.894925: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-31 12:54:36.977234: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpjo2y6ts0
2024-10-31 12:54:36.995

Model successfully converted to TFLite format and saved as 'transmodel_converted1.tflite'.


In [10]:
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="transmodel_converted1.tflite")
interpreter.allocate_tensors()

# Get details of all layers
for layer in interpreter.get_tensor_details():
    print("Layer name:", layer['name'])
    print("Shape:", layer['shape'])
    print("Data type:", layer['dtype'])
    print("-" * 40)


Layer name: serving_default_args_0:0
Shape: [  1 128   4]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant
Shape: [2]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant1
Shape: [ 32 128]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant2
Shape: [32]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant3
Shape: [384 128]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant4
Shape: [384]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant5
Shape: [ 32 128]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant6
Shape: [32]
Data type: <class 'numpy.float32'>
----------------------------------------
Layer name: arith.constant7
Shape: [384 

In [11]:
    model = TransModel()
    model.eval()

TransModel(
  (input_proj): Sequential(
    (0): Conv1d(4, 32, kernel_size=(3,), stride=(1,), padding=same)
    (1): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=same)
    (2): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=same)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear1): Linear(in_features=128, out_features=32, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=32, out_features=128, bias=True)
    (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
  )
  (encoder): TransformerEncoderWAttention(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
      

In [ ]:
def print_detailed_model_layers(model, input_sample):
    print("Detailed Model Structure with Intermediate Outputs:\n" + "=" * 60)
    
    def forward_hook(module, input, output):
        layer_name = module.__class__.__name__
        print(f"Layer Type: {layer_name}")
        if hasattr(module, 'weight') and module.weight is not None:
            print(f"  Weight shape: {tuple(module.weight.shape)}")
        if hasattr(module, 'bias') and module.bias is not None:
            print(f"  Bias shape: {tuple(module.bias.shape)}")
        print(f"  Input shape: {tuple(input[0].shape)}")
        print(f"  Output shape: {tuple(output.shape)}")
        print("-" * 60)

    hooks = []
    for layer in model.modules():
        if len(list(layer.children())) == 0:  # Only leaf layers
            hooks.append(layer.register_forward_hook(forward_hook))
    
    with torch.no_grad():
        model(input_sample, torch.zeros(input_sample.shape[0], 128, 32, 3))  # Sample forward pass
    
    for hook in hooks:
        hook.remove()
        
if __name__ == "__main__":
   
    model = TransModel()

   
    input_sample = torch.randn(1, 128, 4) 

  
    print_detailed_model_layers(model, input_sample)

Detailed Model Structure with Intermediate Outputs:
Layer Type: Conv1d
  Weight shape: (32, 4, 3)
  Bias shape: (32,)
  Input shape: (1, 4, 128)
  Output shape: (1, 32, 128)
------------------------------------------------------------
Layer Type: Conv1d
  Weight shape: (64, 32, 3)
  Bias shape: (64,)
  Input shape: (1, 32, 128)
  Output shape: (1, 64, 128)
------------------------------------------------------------
Layer Type: Conv1d
  Weight shape: (32, 64, 3)
  Bias shape: (32,)
  Input shape: (1, 64, 128)
  Output shape: (1, 32, 128)
------------------------------------------------------------
Layer Type: LayerNorm
  Weight shape: (128,)
  Bias shape: (128,)
  Input shape: (32, 1, 128)
  Output shape: (32, 1, 128)
------------------------------------------------------------
Layer Type: Dropout
  Input shape: (32, 1, 128)
  Output shape: (32, 1, 128)
------------------------------------------------------------
Layer Type: LayerNorm
  Weight shape: (128,)
  Bias shape: (128,)
  Input